# Import model 

In [1]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 6.2 MB/s 
     |████████████████████████████████| 4.2 MB 36.6 MB/s 
     |████████████████████████████████| 1.2 MB 53.0 MB/s 
     |████████████████████████████████| 86 kB 5.7 MB/s 
     |████████████████████████████████| 6.6 MB 46.5 MB/s 
     |████████████████████████████████| 596 kB 62.4 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=807b80d10aceda0d27a4aa9df32314e42bd28593b4790acc8cd4e7e93f37a9bb
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 10.7 MB/s 
     |████████████████████████████████| 235 kB 53.5 MB/s 
     |████████████████████████████████| 581 kB 55.4 MB/s 
     |████████████████████████████████| 965 kB 55.1 MB/s 


In [3]:
from sentence_transformers import util
import torch

from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

In [4]:
from underthesea import sent_tokenize
import json

In [5]:
def find_top_k(top_k, model, question, corpus, corpus_embeddings):
  k = min(top_k, len(corpus))
  query_embedding = model.encode(question, convert_to_tensor=True)

  cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
  # print("error")
  # cos_scores = cos_scores.cpu()

  top_results = torch.topk(cos_scores, k=k)

  return top_results[1]

In [6]:
from copy import deepcopy
def update_top_10(train_data, k):
  copied_data = deepcopy(train_data)
  for topic in copied_data['data']:
    for paragraph in topic['paragraphs']:
      # paragraph['context'] = sent_tokenize(paragraph['context'])
      context = paragraph['context']
      context_embeddings = model.encode(context, convert_to_tensor=True)
      for qa in paragraph['qas']:
        question = qa['question']

        ## Get top k most relevant sentences
        top_k_candidates = find_top_k(k, model, question, context, context_embeddings)
        qa['candidate_indices'] = top_k_candidates.tolist()
  return copied_data

# Process data

In [7]:
import numpy as np

In [8]:
corpus = ['Cô giáo đang ăn kem.',
          'Cô giáo đang ăn bánh mì.',
          'Chị gái đang thử món thịt dê.',
          'Một anh trai đang cưỡi ngựa.',
          'Kị binh đang đi tuần.',
          'Một vụ tai nạn thảm khốc vừa xảy ra.',
          'Đã có ít nhất hai người chết trong vụ xe khách rơi xuống vực.',
          'Một con khỉ đang diễn xiếc.',
          'Một người đàn ông trong bộ đồ tinh tinh đang làm trò.',
          'Lũ quét đột ngột gây thiệt hại cho ít nhất 3 tỉnh miền núi']

corpus_embeddings = model.encode('Cô giáo đang ăn kem.', convert_to_tensor=True)
corpus_embeddings.shape

torch.Size([768])

In [9]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [10]:
%cd "/content/drive/MyDrive/vdt_dsai/dataset"

/content/drive/MyDrive/vdt_dsai/dataset


In [11]:
!ls

test_file.json	train_file.json  val_file.json


# Train data

In [12]:
with open("test_file.json") as f:
  data =  json.load(f)

In [13]:
new_data_train = update_top_10(data, 10)
json_object = json.dumps(new_data_train, ensure_ascii=False)
# with open("xlm_top_10.json", "w") as write_file:
#   write_file.write(json_object)


In [14]:
def find_sentence(corpus, start_answer):
    '''
    Find sentence contains answer
    * corpus: list sentences of context
    * start_answer: the begin position of answer in context
    '''
    start, end = 0, -1
    sent_idx = -1
    for i in range(len(corpus)):
        sentence = corpus[i]
        start = end + 1
        end = start+len(sentence)
        if start<=start_answer<end:
            sent_idx = i
            break
        elif start_answer == end:
            sent_idx = i + 1
            break 
    return sent_idx

def accuracy(data, top_k):
    """
    Calculate accuracy 
    Given queries Q = {Q1, ..., Qm} 
    Document D = {S1, ..., Sn}
    The sentence containing the answer to the question Qi is Ai
    
    Acc@K = 1/|Q| * sum(is Ai in get_top_k(Q, D))
    """
    results = []
    invalid = 0 # number of invalid qa
    for topic in data['data']:
        for paragraph in topic['paragraphs']:
            context = paragraph['context']
            for qa in paragraph['qas']:
                if len(qa['answers']) > 0:

                    # If the answer is from multiple sentences then discard this qa
                    if len(sent_tokenize(qa['answers'][0]['text'])) > 1:
                        invalid += 1
                        continue 

                    start_answer = qa['answers'][0]['answer_start']
                    k = min(top_k, len(qa['candidate_indices']))

                    # Find the index of sentence that contains the answer 
                    ans_sent_idx = find_sentence(context, start_answer)
                    # Check if the above sentence is retrived in top_k 
                    results.append(ans_sent_idx in qa['candidate_indices'][:k])
    return {"top_k":top_k, "# valid qa":len(results), "# invalid qa": invalid,  "true":sum(results), "accuracy":round(sum(results) / len(results), 3)}

In [15]:
print("XLM Test")
vireader_test_results = []
for i in range(10):
    json_i = accuracy(new_data_train, i+1)
    vireader_test_results.append(json_i)
import pandas as pd 
tfidf_test_df = pd.read_json(json.dumps(vireader_test_results))
tfidf_test_df

XLM Test


,top_k,# valid qa,# invalid qa,true,accuracy
0,1,1701,3,1226,0.721
1,2,1701,3,1480,0.870
2,3,1701,3,1597,0.939
3,4,1701,3,1642,0.965
4,5,1701,3,1671,0.982
5,6,1701,3,1689,0.993
6,7,1701,3,1697,0.998
7,8,1701,3,1700,0.999
8,9,1701,3,1700,0.999
9,10,1701,3,1701,1.000
